In [9]:
## Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from ast import literal_eval

In [10]:
DATASET_NAME = 'Amazon-Google'

In [11]:
class LSH :
  
    # Random limits, const
    __max_rand =  1000
    __min_rand = -1000
    
    # Constructor
    def __init__(self, k, L, embedding_size = 150):
        
        # Number of hash function
        self.k = k
        
        # Number of attempts
        self.L = L
        
        # Embedding length 
        self.embedding_size = embedding_size
        
        # Random matrices
        self.normalized_random_matrices = []
        
        for i in range(self.L):
            random_matrix = np.random.randint(self.__min_rand, self.__max_rand,(self.k, self.embedding_size));
            
            # Append normalized random matrices
            self.normalized_random_matrices.append(normalize(random_matrix, axis=1, norm='l1'))
        
    
    # Locality Sensitive hash function
    def locality_sensitive_hash(self, embedding, matrix_index):
        out = 0
      
        for h in self.normalized_random_matrices[matrix_index]:
            if (np.dot(h, embedding) >= 0):
                out = (out << 1) | 1
            else:
                out = (out << 1) | 0

        return out
      
    # Divide in buckets using L-th matrix
    def divide_in_buckets(self, embeddings, matrix_index):
        out = {}
        for embedding in embeddings:
            hash = self.locality_sensitive_hash(embedding, matrix_index)
            if (hash in out):
                out[hash].append(embedding)
            else:
                out[hash] = [embedding]
            
        return out  


In [12]:
test = LSH(k=5, L=2, embedding_size=10)
print(test.normalized_random_matrices)

print(test.locality_sensitive_hash([1,2,3,4,5,6,7,8,9,10], 1))

[array([[-0.15167666,  0.00644015,  0.10393071, -0.16389074,  0.11214746,
        -0.03131246,  0.17876971,  0.03042416,  0.12391739, -0.09749056],
       [-0.18128079,  0.03842365,  0.1235468 ,  0.01044335,  0.18699507,
         0.02246305,  0.07211823, -0.08866995, -0.14857143,  0.12748768],
       [ 0.07494759,  0.1192348 , -0.08883648,  0.00602725, -0.01860587,
        -0.24790356,  0.08018868, -0.24502096,  0.07311321,  0.04612159],
       [-0.06517427, -0.01700198, -0.11561349,  0.04222159,  0.14735052,
         0.04675545, -0.0034004 , -0.26268065, -0.24907906, -0.05072258],
       [-0.08868759, -0.11986518,  0.07099221,  0.10427639, -0.09711397,
        -0.19696651,  0.10722562, -0.11775858,  0.02106594, -0.07604803]]), array([[-0.1010929 , -0.14025501, -0.01930783,  0.1564663 ,  0.09763206,
         0.04462659, -0.10637523,  0.15664845,  0.07723133, -0.1003643 ],
       [-0.07303036, -0.19845207, -0.07402262,  0.19567374, -0.19071244,
        -0.00297678, -0.06965668,  0.05715

In [13]:
## almost equals
embeddings = [[1.2345,2,3,4,5,6,10.4,8,9,10],[1,2,3,4,5,6,7,8,9,10],[1,2,3,5,5,6,7,8,9,10]]
print(len(test.divide_in_buckets(embeddings, 1)))

## not equals
embeddings = np.random.randint(-10000, 10000,(10000, 10))
print(len(test.divide_in_buckets(embeddings, 1)))


1
32


In [14]:
## TEST BLOCKING PERFORMANCE
#    Basta che per ogni tupla vado a prendere la sua corrispondente, ne calcolo
#     i vari L hash e controllo che almeno uno sia uguale e incremento un
#     contatore. La precisione è contatore/numero di tuple controllate, giusto?
def performance_test(filtered_dataset, k, L, embedding_size):
    
    match_found = 0
    
    lsh = LSH(k, L, embedding_size)
    
    # for each elemt in dataset
    for index, row in filtered_dataset.iterrows():
        x_embedding = np.array(literal_eval(row['left_table']))
        y_embedding = np.array(literal_eval(row['right_table']))
          
        x_hashs = set()
        y_hashs = set()
        for i in range(L):
            x_hashs.add(lsh.locality_sensitive_hash(x_embedding, i))
            y_hashs.add(lsh.locality_sensitive_hash(y_embedding, i))
        
        if (len(set.intersection(x_hashs, y_hashs)) > 0):
            match_found += 1
  
    
    return match_found / len(filtered_dataset.index)

In [15]:
## Open dataset 
df = pd.read_csv('../../lsh-test-data/' + DATASET_NAME + '-embeddings.csv')

## Remove 0 labled
df = df[df.label == 1]

precision_max = 0
k_max = 0
L_max = 0
for k in range(10):
    for L in range(10):
        precision = performance_test(df, k + 1, L + 1, 150)
        print("K: {0}, L: {1}, Precision:{2}".format(k + 1, L + 1, precision))
        if (precision >= precision_max):
            precision_max = precision
            k_max = k + 1
            L_max = L + 1

print("Max precision: {0}, k: {1}, L: {2}".format(precision_max, k_max, L_max))

K: 1, L: 1, Precision:0.9316239316239316
K: 1, L: 2, Precision:0.8888888888888888
K: 1, L: 3, Precision:0.9957264957264957
K: 1, L: 4, Precision:1.0
K: 1, L: 5, Precision:1.0
K: 1, L: 6, Precision:1.0
K: 1, L: 7, Precision:1.0
K: 1, L: 8, Precision:1.0
K: 1, L: 9, Precision:1.0
K: 1, L: 10, Precision:1.0
K: 2, L: 1, Precision:0.5384615384615384
K: 2, L: 2, Precision:0.8803418803418803
K: 2, L: 3, Precision:0.9529914529914529
K: 2, L: 4, Precision:0.9829059829059829
K: 2, L: 5, Precision:0.9957264957264957
K: 2, L: 6, Precision:1.0
K: 2, L: 7, Precision:1.0
K: 2, L: 8, Precision:1.0
K: 2, L: 9, Precision:1.0
K: 2, L: 10, Precision:1.0
K: 3, L: 1, Precision:0.3974358974358974
K: 3, L: 2, Precision:0.6239316239316239
K: 3, L: 3, Precision:0.8205128205128205
K: 3, L: 4, Precision:0.905982905982906
K: 3, L: 5, Precision:0.9829059829059829
K: 3, L: 6, Precision:0.9957264957264957
K: 3, L: 7, Precision:1.0
K: 3, L: 8, Precision:1.0
K: 3, L: 9, Precision:1.0
K: 3, L: 10, Precision:1.0
K: 4, L: